In [1]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel

In [2]:
# input and output folders
labeledData = "ex_dataValidation/labeledData.csv"
outputPath = "predictionsLRPipelineValidation/"

In [3]:
# Create a DataFrame from labeledData.csv
# Training data in raw format
labeledDataDF = spark.read.load(labeledData,\
                     format="csv",\
                     header=True,\
                     inferSchema=True)

In [4]:
#labeledDataDF.show()

In [5]:
# Split labeled data in training and test set
# training data : 75%
# test data: 25%
trainDF, testDF = labeledDataDF.randomSplit([0.75, 0.25], seed=10)

In [6]:
#trainDF.show()
#trainDF.count()

In [8]:
#testDF.show()
#testDF.count()

In [9]:
# *************************
# Training step
# *************************

In [10]:
# Define an assembler to create a column (features) of type Vector 
# containing the double values associated with columns attr1, attr2, attr3
assembler = VectorAssembler(inputCols=["attr1", "attr2", "attr3"],\
                            outputCol="features")

In [11]:
# Create a LogisticRegression object.  
# LogisticRegression is an Estimator that is used to 
# create a classification model based on logistic regression.
lr = LogisticRegression()

In [12]:
# We can set the values of the parameters of the 
# Logistic Regression algorithm using the setter methods.
# There is one set method for each parameter
# For example, we are setting the number of maximum iterations to 10
# and the regularization parameter. to 0.0.1
lr.setMaxIter(10)
lr.setRegParam(0.01)

LogisticRegression_c8032462b5c8

In [13]:
# Define a pipeline that is used to create the logistic regression
# model on the training data. The pipeline includes also 
# the preprocessing step
pipeline = Pipeline().setStages([assembler, lr])

In [14]:
# Execute the pipeline on the training data to build the 
# classification model
classificationModel = pipeline.fit(trainDF)

In [15]:
# Now, the classification model can be used to predict the class label
# of new unlabeled data

In [16]:
# Make predictions on the test data using the transform() method of the 
# trained classification model transform uses only the content of 'features' 
# to perform the predictions. The model is associated with the pipeline and hence
# also the assembler is executed
predictionsDF = classificationModel.transform(testDF)

In [17]:
#predictionsDF.printSchema()
#predictionsDF.show()

In [18]:
# The predicted value is column prediction
# The actual label is column label
myEvaluatorAcc = MulticlassClassificationEvaluator(labelCol="label",\
                                                 predictionCol="prediction",\
                                                 metricName='accuracy')

myEvaluatorF1 = MulticlassClassificationEvaluator(labelCol="label",\
                                                 predictionCol="prediction",\
                                                 metricName='f1')


myEvaluatorWeightedPrecision = MulticlassClassificationEvaluator(labelCol="label",\
                                                 predictionCol="prediction",\
                                                 metricName='weightedPrecision')


myEvaluatorWeightedRecall = MulticlassClassificationEvaluator(labelCol="label",\
                                                 predictionCol="prediction",\
                                                 metricName='weightedRecall')

In [19]:
# Apply the evaluators on the predictions associated with the test data

print("Accuracy on test data ", myEvaluatorAcc.evaluate(predictionsDF))
print("F1 on test data ", myEvaluatorF1.evaluate(predictionsDF))
print("Weighted recall on test data ",\
      myEvaluatorWeightedRecall.evaluate(predictionsDF))
print("Weighted precision on test data ",\
      myEvaluatorWeightedPrecision.evaluate(predictionsDF))

Accuracy on test data  0.8148148148148148
F1 on test data  0.7316704459561602
Weighted recall on test data  0.8148148148148148
Weighted precision on test data  0.6639231824417009


In [20]:
# We compute the prediction quality also for the training data.
# To check if the model is overfitted on the training data

In [21]:
# Make predictions on the training data using the transform() method of the 
# trained classification model transform uses only the content of 'features' 
# to perform the predictions. The model is associated with the pipeline and hence
# also the assembler is executed
predictionsTrainingDF = classificationModel.transform(trainDF)

In [22]:
# Apply the evaluators on the predictions associated with the training data
# Print the results on the standard output
print("Accuracy on training data ",\
      myEvaluatorAcc.evaluate(predictionsTrainingDF))
print("F1 on training data ",\
      myEvaluatorF1.evaluate(predictionsTrainingDF))
print("Weighted recall on training data ",\
      myEvaluatorWeightedRecall.evaluate(predictionsTrainingDF))
print("Weighted precision on training data ",\
      myEvaluatorWeightedPrecision.evaluate(predictionsTrainingDF))

Accuracy on training data  0.9178082191780822
F1 on training data  0.9085887598923392
Weighted recall on training data  0.9178082191780822
Weighted precision on training data  0.925395152792413
